**Image Frame Analysis**

Import the necessary libraries

In [2]:
!pip install transformers
!pip install torch torchvision
!pip install Pillow
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.8 MB/s eta 0:00:00


In [3]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
from sentence_transformers import SentenceTransformer, util

Load the BLIP model and processor

In [4]:
# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Load sentence transformers

In [5]:
text_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating a caption for a local image

In [19]:
def generate_caption(image_path):
    # Open the image
    image = Image.open(image_path)

    # Process the image
    inputs = processor(image, return_tensors="pt")

    # Generate the caption
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)

    return caption


In [20]:
def generate_caption_from_url(image_url):
    # Load image from the URL
    image = Image.open(requests.get(image_url, stream=True).raw)

    # Reuse the local image captioning function
    return generate_caption(image)


Checking the caption against the text provided by the user

In [21]:
def check_complaint_match(complaint_text, image_path):
    # Generate a caption for the image
    image_caption = generate_caption(image_path)

    # Calculate similarity between complaint and image caption
    complaint_embedding = text_model.encode(complaint_text, convert_to_tensor=True)
    caption_embedding = text_model.encode(image_caption, convert_to_tensor=True)
    similarity_score = util.pytorch_cos_sim(complaint_embedding, caption_embedding).item()

    # Return results
    return image_caption, similarity_score


In [22]:
def caption_from_local_image(image_path):
    return generate_caption(image_path)


In [30]:
# Test the function with a sample local image
image_path = '/content/img_1.jpg'  # Replace with your local image path
caption = caption_from_local_image(image_path)
print(f"Generated Caption: {caption}")



Generated Caption: a sink filled with dirty water and garbage


In [32]:
complaint_text = "this is the state of the sink on the train"

In [33]:
# Check if the complaint matches the image
image_caption, similarity = check_complaint_match(complaint_text, image_path)
print(f"Complaint: {complaint_text}")
print(f"Generated Caption: {image_caption}")
print(f"Similarity Score: {similarity}")


Complaint: this is the state of the sink on the train
Generated Caption: a sink filled with dirty water and garbage
Similarity Score: 0.5497106313705444


**Video Frame Analysis**

Import necessary libraries

In [26]:
import cv2
from PIL import Image
import torch

Extract videos from video

In [37]:
import cv2
from PIL import Image

def extract_frame_from_video(video_path, start_second=2, interval=5):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the frames per second (fps) of the video
    fps = video.get(cv2.CAP_PROP_FPS)

    # Calculate the frame number to start extracting from the start_second
    frame_number = int(fps * start_second)
    frames = []

    while True:
        ret, frame = video.read()  # Read the next frame

        if not ret:
            break  # Exit the loop when the video ends

        # Process frame at 2nd second, 7th second, 12th second, and so on
        if video.get(cv2.CAP_PROP_POS_FRAMES) == frame_number:
            # Convert frame (which is in BGR format) to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Convert to PIL Image
            pil_image = Image.fromarray(frame_rgb)
            frames.append(pil_image)

            # Update to extract the next frame at (current second + interval)
            frame_number += int(fps * interval)

    video.release()
    return frames


Caption from frame

In [38]:
def caption_from_frame(frame):
    # Process the image and generate a caption
    inputs = processor(frame, return_tensors="pt")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

Test with a local video

In [39]:
# Test with a local video
video_path = '/content/stock video.mp4'  # Replace with your video path
frames = extract_frame_from_video(video_path, interval=5)

# Process each extracted frame
for i, frame in enumerate(frames):
    caption = caption_from_frame(frame)
    print(f"Generated Caption for frame {i+1}: {caption}")


Generated Caption for frame 1: a man is walking through a trash filled hallway
Generated Caption for frame 2: a bathroom with a sink and a trash can
Generated Caption for frame 3: a dirty toilet with a broken lid and a trash can
